In [66]:
import xgboost as xgb
import pandas as pd
import cudf
import numpy as np

In [67]:
df = cudf.read_parquet('test.parquet')

In [68]:
df = df.set_index('customer_ID')

In [69]:
df = cudf.get_dummies(df, dtype= 'Int8')

In [70]:
df = df.astype('float32')

In [71]:
df = df.fillna(0)

In [72]:
# Load the XGBoost model from the file
xgb_model = xgb.XGBRFRegressor()
xgb_model.load_model('xgboost_model.json')

In [73]:
df_numpy = df.to_pandas().values

In [74]:
# Get the list of features expected by the model
expected_features = xgb_model.get_booster().feature_names
print("Expected Features:", expected_features)

Expected Features: None


In [75]:
# Get the list of features expected by the model
expected_features = xgb_model.get_booster().feature_names
print(expected_features)
# Get the list of features present in the DataFrame
df_features = df.columns.tolist()
print(df_features)

None
['P_2', 'B_1', 'B_2', 'R_1', 'S_3', 'B_3', 'D_43', 'D_44', 'B_4', 'D_45', 'R_2', 'D_47', 'D_48', 'B_7', 'B_8', 'B_9', 'R_3', 'B_11', 'S_7', 'D_55', 'D_58', 'D_61', 'D_62', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_22', 'D_70', 'B_23', 'D_74', 'D_75', 'D_77', 'D_78', 'B_30', 'B_33', 'S_25', 'B_37', 'B_38']


In [77]:
preds = xgb_model.predict(df.values)

/home/simon/miniconda3/envs/rapids-24.08/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [13:52:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cpu, while the input data is on: cuda:0.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [78]:
binary_preds = (preds > 0.5).astype(int)

In [80]:
result_df = cudf.DataFrame({
    'customer_ID': df.index.to_pandas(),  # Convert index to pandas for compatibility
    'prediction': binary_preds
})

In [81]:
# Export the result DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)